# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
#!pip install pymongo

In [4]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [7]:
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [10]:
companies = db.get_collection('companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [16]:
proj = {'_id' : 0, 'name' : 1}
companies.find_one({'name' : 'Babelgum'}, proj)

{'name': 'Babelgum'}

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [21]:
proj2 = {'_id' : 0, 'name' : 1, 'number_of_employees' : 1}
high_employ = list(  companies.find({'number_of_employees' : {'$gt': 5000} }, proj2).limit(20).sort('number_of_employees', -1)  )
high_employ
#El código que se pide sería sin la proj2

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [35]:
proj3 = {'_id' : 0, 'name' : 1, 'founded_year' : 1}
fundacion_altos = list(companies.find({'$and' : [{'founded_year' : {'$gte' : 2000}}, {'founded_year' : {'$lte' : 2005}}]}, proj3).limit(5).sort('founded_year', -1))
fundacion_bajos = list(companies.find({'$and' : [{'founded_year' : {'$gte' : 2000}}, {'founded_year' : {'$lte' : 2005}}]}, proj3).limit(5).sort('founded_year', 1))
print(fundacion_bajos)
print(fundacion_altos)
#este sería el código sin restricciones (el que se pide)
fundacion= list(companies.find({'$and' : [{'founded_year' : {'$gte' : 2000}}, {'founded_year' : {'$lte' : 2005}}]}))

[{'name': 'MeeVee', 'founded_year': 2000}, {'name': 'Steorn', 'founded_year': 2000}, {'name': 'Boonex', 'founded_year': 2000}, {'name': 'PhotoBox', 'founded_year': 2000}, {'name': 'AllofMP3', 'founded_year': 2000}]
[{'name': 'Omnidrive', 'founded_year': 2005}, {'name': 'Zoho', 'founded_year': 2005}, {'name': 'Jingle Networks', 'founded_year': 2005}, {'name': 'Helio', 'founded_year': 2005}, {'name': 'Wetpaint', 'founded_year': 2005}]


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [55]:
proj4 = {'_id' : 0, 'name' : 1, 'ipo' : 1}
valuation = list(companies.find({'$and' : [{'ipo.valuation_amount' : {'$gt' : 100000000}}, {'founded_year' : {'$lte' : 2010}}]}, proj4))
valuation

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Brightcove',
  'ipo': {

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [49]:
proj5 = {'_id' : 0, 'name' : 1, 'number_of_employees' : 1, 'founded_year' : 1}
#Código sin abreviaturas en el output
little = list(companies.find({'$and' : [{'number_of_employees' : {'$lt':1000}}, {'founded_year' : {'$gt' : 2005}}]}).limit(10).sort('number_of_employees', 1))
little_little = list(companies.find({'$and' : [{'number_of_employees' : {'$lt':1000}}, {'founded_year' : {'$gt' : 2005}}]}, proj5).limit(10).sort('number_of_employees', 1))
little_little

[{'name': 'CastTV', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Mashery', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Cofoundr', 'number_of_employees': 0, 'founded_year': 2007},
 {'name': 'Joost', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'EQO', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Go2Web20', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Sportingo', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Gumiyo', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'Checkster', 'number_of_employees': 0, 'founded_year': 2006},
 {'name': 'coRank', 'number_of_employees': 0, 'founded_year': 2007}]

### 6. All the companies that don't include the `partners` field.

In [54]:
sinpartners = list(companies.find({'partners': {'$exists' : False}}))
sinpartners

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [59]:
none_code = list( companies.find({'category_code' : None}) )
#none_code

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [70]:
proj6 = {'_id' : 0, 'name' : 1, 'number_of_employees' : 1}
empresas_bien = list( companies.find({'$and': [{'number_of_employees':{'$gte':100}}, {'number_of_employees': {'$lt':1000}}]},proj6))
empresas_bien

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [81]:
#proj7 = {'_id': 0, 'name': 1, 'ipo.valuation_amount' : 1}
empresas_feten = list(companies.find({'ipo.valuation_amount' : {'$exists': True}}).sort('ipo.valuation_amount', -1))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [86]:
#proj8 = {'_id': 0, 'name': 1, 'number_of_employees' : 1}
empleados = list(companies.find({'number_of_employees' : {'$exists': True}}).sort('number_of_employees', -1).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [89]:
scnd_semester = list(companies.find({'founded_month' : {'$gte': 7}}).limit(1000))
len(scnd_semester)

1000

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [95]:
#proj8 = {'_id': 0, 'name': 1, 'acquisition.price_amount' : 1, 'founded_year' :1}
and1 = {'founded_year' : {'$lt' : 2000}}
and2 = {'acquisition.price_amount' : {'$gte' : 10_000_000}}
precio_adquisicion = list(companies.find({'$and':[and1, and2]}))
len(precio_adquisicion)

209

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [98]:
proj9 = {'_id': 0, 'name': 1, 'acquisition' : 1}
adquiridas = list(companies.find({'acquisition.acquired_year' : {'$gte': 2010}}, proj9).sort('acquisition.price_amount', -1))
len(adquiridas)

1100

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [119]:
proj10 = {'_id': 0, 'name': 1, 'founded_year' :1}
anyo_fundacion = list( companies.find({'founded_year': {'$ne': None}}, proj10).sort('founded_year', 1).limit(10) )
anyo_fundacion

[{'name': 'US Army', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'WeGame', 'founded_year': 1840}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [141]:
#proj11 = {'_id': 0, 'name': 1, 'founded_day' : 1, 'acquisition.price_amount' : 1 }
and_01 = {'founded_day' : {'$lte' : 7}}
and_02 = {'acquisition.price_amount' : {'$ne': None}}
precio_adquisicion = list(companies.find({'$and':[and_01, and_02]}).limit(10).sort('acquisition.price_amount', -1))
len(precio_adquisicion)

10

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [138]:
#proj11 = {'_id': 0, 'name': 1, 'number_of_employees' : 1, 'acquisition.price_amount' : 1 }
empresas_web = list(companies.find({'$and':[{'category_code' : 'web'},{'number_of_employees': {'$gt' : 4000}}]}).sort('number_of_employees', 1))
len(empresas_web)

9

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [145]:
condicion_01 = {'acquisition.price_amount' : {'$ne' : None}}
condicion_02 = {'acquisition.price_amount' : {'$gt' : 10000000}}
condicion_03 = {'acquisition.price_currency_code' : 'EUR'}
adquisiciones_euros = list( companies.find({'$and': [condicion_01, condicion_02, condicion_03]}))
len(adquisiciones_euros)

7

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [152]:
proj12 = {'_id' : 0, 'name' : 1, 'acquisition' : 1}
condicion_04 = {'acquisition.acquired_month' : {'$ne' : None}}
condicion_05 = {'acquisition.acquired_month' : {'$lte' : 3}}
trimestral = list(companies.find({'$and': [condicion_04, condicion_05]}, proj12).limit(10))
trimestral

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [156]:
con_01 = {'founded_year' : {'$gte' : 2000}}
con_02 = {'founded_year' : {'$gte' : 2010}}
con_03 = {'acquisitions.acquired_year': {'$gt': 2011}}

bonus_01 = list(companies.find({'$and': [con_01, con_02, con_03]}))
bonus_01

3

### 20. All the companies that have been 'deadpooled' after the third year.

In [161]:
con_04 = {'deadpooled_year' : {'$ne' : None}}
con_05 = {'deadpooled_year' : {'$gt' : 3}}
bonus_02 = list(companies.find({'$and': [con_04, con_05]}))
len(bonus_02)

926